Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [7]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000 # Number of training examples

graph = tf.Graph() # Instantiate a new tensorflow graph
with graph.as_default(): # Attach the graph

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :]) # tf.constant() makes the datasets visible to the graph
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels])) # Parameters are initialized using tf.Variable()
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases # XW + b
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # ^ Compute the average cross_entropy
    # tf.nn.softmax_cross_entropy_with_logits() returns the cross-entropy computed on the results of softmax(logits)
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # Optimizer object (0.5 = learning rate)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits) # Class probabilities (during training)
    valid_prediction = tf.nn.softmax(        # Class probabilities (for cross-validation)
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases) # Class probabilities (for testing)

The process we went through when setting up the graph was:

0. Instantiated the graph, then attached it. Note that we specified the number of training examples here.
1. Introduce the datasets to the graph via tf.constant()
2. Introduce the model's parameters - W and b - to the graph via tf.Variable() (we also initialized these)
3. Expressed the loss function - this took the softmax probabilities as inputs.
4. Created an optimization object and associated the loss function with it.
5. Defined feedback parameters to display during training.

Let's run this computation and iterate:

In [8]:
num_steps = 801

def accuracy(predictions, labels): # Computes prediction accuracy as a percentage
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
        / predictions.shape[0])

with tf.Session(graph=graph) as session: # Intantiate a TensorFlow session, specifying the graph we made as an input.
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run() # Initialize random variables
    print('Initialized!')
    for step in range(num_steps): # Run the number of iterations we say to
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction]) # .run() steps through one iteration of optimization
        if (step % 100 == 0): # Report the loss function's value and classifier accuracy every 100 steps
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies. (.eval() computes a particular variable's value)
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized!
Loss at step 0: 20.384365
Training accuracy: 6.9%
Validation accuracy: 9.3%
Test accuracy: 9.3%
Loss at step 100: 2.291604
Training accuracy: 71.2%
Validation accuracy: 70.8%
Test accuracy: 78.2%
Loss at step 200: 1.856400
Training accuracy: 73.9%
Validation accuracy: 73.1%
Test accuracy: 80.2%
Loss at step 300: 1.616425
Training accuracy: 75.3%
Validation accuracy: 74.0%
Test accuracy: 81.0%
Loss at step 400: 1.452093
Training accuracy: 76.0%
Validation accuracy: 74.6%
Test accuracy: 81.6%
Loss at step 500: 1.329108
Training accuracy: 77.0%
Validation accuracy: 75.1%
Test accuracy: 82.0%
Loss at step 600: 1.232408
Training accuracy: 77.5%
Validation accuracy: 75.3%
Test accuracy: 82.2%
Loss at step 700: 1.153724
Training accuracy: 77.9%
Validation accuracy: 75.5%
Test accuracy: 82.5%
Loss at step 800: 1.088146
Training accuracy: 78.5%
Validation accuracy: 75.7%
Test accuracy: 82.8%


So after defining our graph, we passsed it to a Session, in which we:
1. Initialized the constants with random values.
2. Stepped through optimization a fixed number of times by calling .run([optimizer, loss, training_predictions]) within each iteration of a for loop.
3. Printed our performance by computing baked-in expressions via .eval()

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [12]:
batch_size = 128 # Specify the number of examples to use at each iteration's loss function computation

graph = tf.Graph() # Instantiate the graph
with graph.as_default(): # Attach the graph

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size)) # We use tf.placeholder() to allow data to be passed at runtime
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) # num_labels is the number of classes
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels])) # Same as before
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases # As before
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # As before, we compute the mean cross-entropy

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # We use a fixed learning rate
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [13]:
num_steps = 3001 # More steps (but faster!) since we're using batch gradient descent

with tf.Session(graph=graph) as session: # Instantiate a session, passing the graph to it
    tf.global_variables_initializer().run() # Initialize random variables
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size) # To vary the minibatch taken
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels} # We use a dictionary to pass values to tf.placeholder()s
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict) # Step through an optimization iteration
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 21.433046
Minibatch accuracy: 6.2%
Validation accuracy: 11.2%
Minibatch loss at step 500: 1.430115
Minibatch accuracy: 78.9%
Validation accuracy: 75.3%
Minibatch loss at step 1000: 0.873820
Minibatch accuracy: 83.6%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 1.534579
Minibatch accuracy: 76.6%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.966453
Minibatch accuracy: 76.6%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 1.012788
Minibatch accuracy: 78.9%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 0.842003
Minibatch accuracy: 82.8%
Validation accuracy: 78.4%
Test accuracy: 86.3%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [ ]:
# Set up the graph.
# 1. Create placeholders for the minibatch training data and labels.
# 2. Create constants for the test data (X) and labels.
# 3. Create variables for the first-layer weights and biases. W is [input_D x 1024], b is [1024] (tf does the broadcasting)
# 4. -> Pass XW + b to nn.relu()
# 5. -> Relu will output an [n_datum x 1024] tensor, X
# 6. -> Compute logits = XW + b, this time with W:[1024 x n_classes], b:[n_classes]
# 7. -> Pass these values (the logits) to tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(train_labels, logits))
# 8. -> Create an optimizer.
# 8. -> Create prediction variables so that we can see how well the network is training.

# Run it
# 0. Specify a batch size.
# 1. Start the session, pass the graph to it.
# 2. Loop for n_steps,
#    creating a new minibatch each time and passing it to the model via feed_dict{},
#    calling [_, l, predictions] = session.run() afterwards each time to run an optimization step.
# 3. Every kth iteration, print the loss, minibatch, and training error.
# 4. On the final iteration, print the test error.

In [21]:
# Set up the graph

batch_size = 256
n_relus    = 1024

graph = tf.Graph();
with graph.as_default():
    # Data handles
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape = (batch_size, num_labels)) # Labels are dummy encoded
    tf_test_data     = tf.constant(test_dataset)
    tf_valid_data    = tf.constant(valid_dataset)
    
    # Layer 1
    weights_1        = tf.Variable(tf.truncated_normal([image_size * image_size, n_relus]))
    biases_1         = tf.Variable(tf.zeros([n_relus]))
    train_out_1      = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    # Layer 2
    weights_2        = tf.Variable(tf.truncated_normal([n_relus, num_labels]))
    biases_2         = tf.Variable(tf.truncated_normal([num_labels]))
    train_out_2      = tf.matmul(train_out_1, weights_2) + biases_2
    
    # Loss calculation
    loss             = tf.reduce_mean(
                        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = train_out_2))
    
    # Optimizer
    tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Training predictions
    training_predictions = tf.nn.softmax(train_out_2)
    
    # Validation predictions
    valid_out_1          = tf.nn.relu(tf.matmul(tf_valid_data, weights_1) + biases_1)
    valid_predictions    = tf.nn.softmax(tf.matmul(valid_out_1, weights_2) + biases_2)
    
    # Testing predictions
    test_out_1           = tf.nn.relu(tf.matmul(tf_test_data, weights_1) + biases_1)
    valid_predictions    = tf.nn.softmax(tf.matmul(test_out_1, weights_2) + biases_2)